The project is about merging 3 dataset Energy, Productions, and Fermate

In [ ]:
from scripts.getDataset import getEntireDataset
from scripts.plots import plot
import pandas as pd

pd.set_option("display.max_rows", None)
pd.options.mode.copy_on_write = False

df = getEntireDataset(306, 2023, 1)

plot(df)

Now we can check for all the dataset and see what we can do with them

In [1]:
from scripts.plots import plot
from scripts.getDataset import getEntireDataset, forEveryMachine
import pandas as pd

pd.set_option("display.max_rows", None)
pd.options.mode.copy_on_write = False


def fn(machineId, year, month):
    df = getEntireDataset(machineId, year, month)

    plot(df)


forEveryMachine(fn)


df = getEntireDataset(306, 2023, 1)

print(df.loc[:, ("TIMESTAMP", "EnergyConsumption", "COD_ART")])

plot(df)

__Getting Fermate__
WARNING, you are dropping STOP_CODE that is not always the same
[10  4]
__Getting Productions__
__Getting Enegy Consumption__


__Getting Fermate__
WARNING, you are dropping STOP_CODE that is not always the same
[10  4]
__Getting Productions__
__Getting Enegy Consumption__


__Getting Fermate__
WARNING, you are dropping STOP_CODE that is not always the same
[ 4 10  9]
__Getting Productions__
__Getting Enegy Consumption__


__Getting Fermate__
WARNING, you are dropping STOP_CODE that is not always the same
[10  4]
__Getting Productions__
WARNING, Productions was Empty
__Getting Enegy Consumption__


__Getting Fermate__
WARNING, you are dropping STOP_CODE that is not always the same
[ 4 10]
__Getting Productions__
__Getting Enegy Consumption__


...

__Getting Fermate__
WARNING, Fermate was Empty
__Getting Productions__
WARNING, Productions was Empty
__Getting Enegy Consumption__


__Getting Fermate__
WARNING, Fermate was Empty
__Getting Productions__
__Getting Enegy Consumption__


__Getting Fermate__
WARNING, Fermate was Empty
__Getting Productions__
__Getting Enegy Consumption__


__Getting Fermate__
WARNING, Fermate was Empty
__Getting Productions__


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe8 in position 211: invalid continuation byte